In [2]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YoutubeContentRepository
from strategy import HuggingFaceLLM
from domain.youtube_timeline_summary import YoutubeTimelineSummary
from domain.youtube_timeline_section import YoutubeTimelineSection
from domain.youtube_content import YouTubeContent

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
content_repo: YoutubeContentRepository = YoutubeContentRepository(client)

# LLM
local_llm = HuggingFaceLLM(model_id="Bllossom/llama-3.2-Korean-Bllossom-AICA-5B", quantization="16bit")


# Content
contents = content_repo.find_all()
content: YouTubeContent = contents[1]

Connected to MongoDB


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use mps


# 아웃 파서

In [3]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# ✅ (핵심 키포인트 모델)
class YoutubeKeyPointModel(BaseModel):
    term: str = Field(..., description="핵심 용어")
    description: str = Field(..., description="핵심 용어에 대한 설명")

# ✅ (핵심 키포인트 컬렉션 모델)
class YoutubeKeyPointCollectionModel(BaseModel):
    key_points: List[YoutubeKeyPointModel] = Field(..., description="영상에서 추출된 핵심 키포인트 리스트")

# ✅ 아웃파서 정의
key_point_parser = PydanticOutputParser(pydantic_object=YoutubeKeyPointCollectionModel)
print(key_point_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"YoutubeKeyPointModel": {"properties": {"term": {"description": "핵심 용어", "title": "Term", "type": "string"}, "description": {"description": "핵심 용어에 대한 설명", "title": "Description", "type": "string"}}, "required": ["term", "description"], "title": "YoutubeKeyPointModel", "type": "object"}}, "properties": {"key_points": {"description": "영상에서 추출된 핵심 키포인트 리스트", "items": {"$ref": "#/$defs/YoutubeKeyPointModel"}, "title": "Key Points", "type": "array"}}, "required": ["key_points"]}
```


# 프롬프트 템플릿

In [4]:
from langchain.prompts import PromptTemplate

key_point_prompt_template = PromptTemplate(
    partial_variables={'output_format': key_point_parser.get_format_instructions()},
    input_variables=["summary"],
    template="""
다음은 유튜브 타임라인 요약 문서입니다. 이 내용을 분석하여 10개의 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.
중요하지 않더라도 알면 좋을 추가 핵심 용어도 더해주세요.

[타임라인 요약]:
{summary}

[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는  10개의 키포인트 리스트로 구성됩니다.
- 반드시 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

[반환 형식]:
{output_format}
"""
)

# LLM

In [5]:
chain = key_point_prompt_template | local_llm | key_point_parser


summary: YoutubeTimelineSummary = content.timeline_summary

summary_texts = f"{summary.text}\n"

for section in summary.sections:
    summary_texts = summary_texts + '\n'
    for text in section.texts:
        summary_texts = summary_texts + f'- {text}\n'


print(key_point_prompt_template)

input_variables=['summary'] input_types={} partial_variables={'output_format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"YoutubeKeyPointModel": {"properties": {"term": {"description": "핵심 용어", "title": "Term", "type": "string"}, "description": {"description": "핵심 용어에 대한 설명", "title": "Description", "type": "string"}}, "required": ["term", "description"], "title": "YoutubeKeyPointModel", "type": "object"}}, "properties": {"key_points": {"description": "영상에서 추출된 핵심 키포인트 리스트", "items": {"$ref": "#/$defs/YoutubeKeyPointModel"}, "title": "Key Points", "type": "array"}},

In [6]:
input_data = {
        "summary": summary_texts
    }

response: YoutubeKeyPointCollectionModel = chain.invoke(input_data)

KeyboardInterrupt: 

In [ ]:
for key_point in response.key_points:
    print(f'{key_point.term}: {key_point.description}')